### 구글드라이브 연결
> - drive 폴더가 붙은 것을 확인한다.
> - drwx------ 4 root root 4.0K Mar 22 11:21 drive

In [1]:
# from google.colab import drive 
# drive.mount('/content/drive/')

### Install / Import 모듈 

In [2]:
# 코랩에 없는 라이브러리 설치해주기
# !pip install bayesian-optimization

In [3]:
import os
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import matplotlib.pyplot as plt             # * 그래프 이미지

from tqdm import tqdm                       # 진행바

from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings           
import random                       
warnings.filterwarnings("ignore")           # 경고 문구 미표시

In [4]:
# 데이터가 많아 5분 정도 걸립니다.  
# 전체경로 = /content/drive/My Drive/Colab Notebooks/competition/c03_starcraft_prediction/data_raw/train.csv
# /content/drive/My Drive/Colab Notebooks/
# dir_base = '/content/drive/My Drive/Colab Notebooks/competition/''

dir_base = '/home/yk/0325_Starcraft/competition/'

raw      = 'c03_starcraft_prediction/data_raw/'
remake   = 'c03_starcraft_prediction/data_remake/'
submit   = 'c03_starcraft_prediction/data_submit/'
                                                                # WAY-01 = LGBM
filename_submit = 'submission_0413_way01_15_75_final.csv'       # WAY-02 = XGBM

### 사용함수의 정의

In [5]:
os.chdir(dir_base)
from _assets.modules import *
from _assets.module_data_preps import *
from _assets.module_lgbm_model import *


# OS 화일 및 DF 정보조회를 위한 탐색 모듈


# GAME_ID 유니크 데이터 전처리를 위한 모듈 - baseline offer


# LGBM_CV 모델 - Dacon Baseline 참조



In [6]:
show_ls(dir_base + remake)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake/
----------------------------------------
  01. .ipynb_checkpoints
  02. accumujated_winning_rate_per_time.csv
  03. accumulated_winning_rate_per_time.csv
  04. df_new_rate04_correct.csv
  05. df_win_rate_every_10sec.csv
  06. df_win_rate_every_20sec.csv
  07. df_win_rate_every_40sec.csv
  08. df_xtest_remake.csv
  09. df_xtest_remake_final.csv
  10. df_xtrain_remake.csv
  11. df_xtrain_remake_final.csv
  12. df_ytrain_remake.csv


In [7]:
%%time
"""
# 데이터 전처리 과정 - 약 7 ~ 10분 정도 걸립니다 ... 여기서 세션 리셋 됨!
# x_train, y_train = data_preparation(train, answer=True)           # [ 67,091,776 x 7 ] ... 70%
# show_infoDF_from(x_train) # *** DATA SHAPE = [ 38,872 x 27 ] ... [ game_id ... ]
# show_infoDF_from(y_train) # *** DATA SHAPE = [ 38,872 x 3 ] .. ['Unnamed: 0', 'game_id', 'winner']
"""

x_train = pd.read_csv(dir_base + remake + 'df_xtrain_remake_final.csv')  # to <class 'pandas.core.frame.DataFrame'>
ytrain = pd.read_csv(dir_base + remake + 'df_ytrain_remake.csv')  # to <class 'numpy.ndarray'>
y_train = ytrain['winner'].values                                 # array([1, 1, 0, ..., 0, 1, 0])

CPU times: user 79.5 ms, sys: 7.91 ms, total: 87.4 ms
Wall time: 87 ms


In [8]:
pd.Series(y_train).value_counts()
# 1    19499
# 0    19373            total = 38,872  y_train = np.array 38,872

1    19499
0    19373
dtype: int64

In [9]:
# 제대로 작성이 됬는지 확인 다 끝났으면, 저장한다. 저장끝났으면 비활성화 처리


"""  # 저장했으면 비활성 처리
df_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')
df_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')
"""

"  # 저장했으면 비활성 처리\ndf_xtrain.to_csv(dir_base + remake + 'df_xtrain_remake.csv')\ndf_ytrain.to_csv(dir_base + remake + 'df_ytrain_remake.csv')\n"

# LGBM Classifier 모델 적용
 -  Light Gradient Boost Method 적용
 - [출처] :  YW & YY's Python, Machine & Deep Learning
 > - Light GBM: A Highly Efficient Gradient Boosting Decision Tree 논문 리뷰
 > - [Light GBM 모델 설명 및 사용법](https://greeksharifa.github.io/machine_learning/2019/12/09/Light-GBM/)

 > 다중 분류, 클릭 예측, 순위 학습 등에 주로 사용되는 Gradient Boosting Decision Tree (GBDT)는 굉장히 유용한 머신러닝 알고리즘이며, XGBoost나 pGBRT 등 효율적인 기법의 설계를 가능하게 하였다. 이러한 구현은 많은 엔지니어링 최적화를 이룩하였지만 고차원이고 큰 데이터 셋에서는 만족스러운 결과를 내지 못하는 경우도 있었다. 왜냐하면 모든 가능한 분할점에 대해 정보 획득을 평가하기 위해 데이터 개체 전부를 스캔해야 했기 때문이다. 이는 당연하게도, 굉장히 시간 소모적이다.


In [10]:
%%time
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 

# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'learning_rate': (0.0001, 0.1),    # learning_rate,    범위(0.0001~0.1)
        'num_leaves': (16, 1024),          # num_leaves,       범위(16~1024)
        'n_estimators': (16, 1024),        # n_estimators,     범위(16~1024)
        'subsample': (0, 1),               # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),        # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),              # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),             # reg_lambda,       범위(0~50)
    }, 
    random_state=random.randrange(50000)   # 시드 고정 = 4321
)

# lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화 = Original Baseline!
lgbBO.maximize(init_points=15, n_iter=75)


"""
# LightGBM Classifier Document =  https://bit.ly/2QGqFTS
# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도 했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
"""

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.652    |  0.6562   |  0.03671  |  277.5    |  251.0    |  2.874    |  28.99    |  0.02663  |
|  2        |  0.6485   |  0.9872   |  0.0433   |  1.009e+0 |  55.61    |  5.47     |  43.95    |  0.8761   |
|  3        |  0.6423   |  0.1752   |  0.05512  |  863.1    |  223.1    |  9.379    |  20.77    |  0.3431   |
|  4        |  0.6396   |  0.1752   |  0.003352 |  829.2    |  203.7    |  9.147    |  25.43    |  0.7816   |
|  5        |  0.6371   |  0.6484   |  0.07355  |  718.8    |  514.9    |  9.648    |  38.06    |  0.875    |
|  6        |  0.6421   |  0.8293   |  0.09185  |  184.2    |  352.4    |  3.074    |  20.52    |  0.06241  |
|  7        |  0.6483   |  0.2422   |  0.05864  |  471.1    |  73.11    |  2.068    |  11.15    |  0.7492   |
|  8      

|  45       |  0.6532   |  0.3673   |  0.02692  |  315.7    |  864.5    |  6.504    |  48.78    |  0.3793   |
|  46       |  0.6493   |  0.4321   |  0.00187  |  890.4    |  924.7    |  2.315    |  1.815    |  0.481    |
|  47       |  0.6464   |  0.1966   |  0.08063  |  1.004e+0 |  23.88    |  5.728    |  1.488    |  0.927    |
|  48       |  0.6472   |  0.06618  |  0.04069  |  892.2    |  20.69    |  3.52     |  44.83    |  0.3976   |
|  49       |  0.6458   |  0.2897   |  0.03932  |  217.5    |  243.0    |  0.4798   |  0.193    |  0.8839   |
|  50       |  0.6393   |  0.1786   |  0.009407 |  163.7    |  509.9    |  1.241    |  48.69    |  0.1831   |
|  51       |  0.651    |  0.7626   |  0.02771  |  441.5    |  633.5    |  1.57     |  49.25    |  0.7582   |
|  52       |  0.6501   |  0.3032   |  0.003053 |  897.7    |  460.3    |  1.462    |  0.3586   |  0.786    |
|  53       |  0.6493   |  0.203    |  0.04342  |  334.0    |  457.6    |  1.664    |  47.87    |  0.8564   |
|  54     

|  90       |  0.6329   |  0.4314   |  0.09001  |  940.2    |  370.4    |  1.163    |  2.317    |  0.716    |
CPU times: user 12h 18min 25s, sys: 6min 54s, total: 12h 25min 19s
Wall time: 46min 52s


'\n# LightGBM Classifier Document =  https://bit.ly/2QGqFTS\n# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도 했습니다.\n# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!\n'

### 3.0 모델 학습 및 검증
- Model Tuning & Evaluation

> 1. AUC가 가장 높은 하이퍼 파라미터를 사용해 최종 모델을 얻습니다.
> 1. 훈련 세트와 같은 방법으로 테스트 세트에서 Feature를 추출합니다.
> 1. 최종 모델을 사용해 예측을 수행합니다.
> 1. 예측 결과를 submission.csv로 저장합니다.

In [11]:
params = lgbBO.max['params']

models = lgb_cv(
        params['num_leaves'], 
        params['learning_rate'], 
        params['n_estimators'], 
        params['subsample'], 
        params['colsample_bytree'], 
        params['reg_alpha'], 
        params['reg_lambda'], 
        x_data=x_train, 
        y_data=y_train, 
        n_splits=5, 
        output='model',
    )

# TEST를 풀기위해, 학습데이터로 변경 (data_preps)
 1. Train Data set = [ 67,091,776 x 7 ] ... 70%
  - game_id 별로 정리하면 [38,872 x 27]로 압축된다. 
  - 경기당, 평균 1,726개의 이벤트가 존재한다.(압축률 0.057939 %)
 > - x_train set = *** DATA SHAPE = [ 38,872 x 27 ] pandas.DataFrame
 > - y_train set = *** DATA SHAPE = [ 38,872 x 1 ]  np.array
 
 1. Test Data set  = [ 28,714,849 x 6 ] ... 30%
  - game_id 별로 정리하면 [16,787 x 27] 로 압축된다. 
  - 경기당 평균 1711개의 이벤트가 존재한다.(압축률 0.058461 %)
 > - x_test set = *** DATA SHAPE = [16,787 x 27] pandas.DataFrame 
 > - y_test set = *** DATA SHAPE = [16,787 x 1] np.array =  대회주관자가 가지고 있음
 
 1. Submission = y_test set 
  - submission = [16,787 x 1] np.array to [16,787 x 2 ] pandas.DataFrame --> df.csv

In [12]:
"""
# 테스트용 데이터를 전처리 해서 분석 준비 ... (시간소요) 미리 저장한 화일을 불러온다
# 저장했으면 비활성 처리
# x_test, _ = data_preparation(test, answer=False)                 # [ 28,714,849 x 6 ] ... 30%
# x_test.to_csv(dir_base + remake + 'df_xtest_remake.csv')
# x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake.csv')  # [  16,787 x 27 ]
"""
x_test = pd.read_csv(dir_base + remake + 'df_xtest_remake_final.csv')    # *** DATA SHAPE = [ 16,787 x 27 ]


# 예측결과 Submission 화일 만들기

In [13]:
"""
# pd.read_csv('data/sample_submission.csv', index_col=0)
# sample_submission = pd.read_csv(dir_base + sub_base + "sample_submission.csv")
"""

preds = []

for model in models:
    pred = model.predict_proba(x_test)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis=0)

In [14]:
show_ls(dir_base + raw)

False
DIR_TARGET=/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_remake
/home/yk/0325_Starcraft/competition/c03_starcraft_prediction/data_raw/
----------------------------------------
  01. sample_submission.csv
  02. test.csv
  03. train.csv


In [15]:
submission = pd.read_csv(dir_base + raw + "sample_submission.csv") 


submission['winner'] = pred
submission.to_csv(dir_base + submit + filename_submit, index=False)

# 결과 확인하기
- 3월 데이콘 대회 홈페이지 = https://bit.ly/39bqWVg
- 결과 제출하기 = https://dacon.io/competitions/official/235583/mysubmission/

In [16]:
df_1 = submission
df_1

,game_id,winner
0,38872,0.644543
1,38873,0.494188
2,38874,0.412015
3,38875,0.204855
4,38876,0.438289
...,...,...
16782,55654,0.692170
16783,55655,0.318288
16784,55656,0.767258
16785,55657,0.511795
